In [ ]:
--

In [1]:
import pandas as pd
from gensim import corpora
import os

In [4]:
# 创建预训练的文件夹
PRETRAIN_DIR = '../../user_data/pre_train_model'
if not os.path.exists(PRETRAIN_DIR):
    os.makedirs(PRETRAIN_DIR)
print('已创建预训练文件夹')

In [5]:
def readText(filename):
    '''读取文本'''
    with open(filename, encoding='gbk') as f:
        content = f.readlines()
    return ''.join([x.replace('\n','||') for x in content])

预料准备

In [20]:
paths = ['../../data/train/txt/','../../data/test/txt/','../../data/sample_example/']
corpus = []
for path in paths:
    filenames = os.listdir(path)
    filenames = [x for x in filenames if 'txt' in x]
    for filename in filenames:
#         print(path + filename)
        with open(path + filename,encoding='gbk') as f:
            content = f.readlines()
            content = ''.join(content).replace('\n','||').split('||||')
            corpus += content
corpus = [x.replace('||','\n') for x in corpus]

In [34]:
with open('corpora/book_review_bert.txt','w+') as f:
    for line in corpus:
        f.write(line+'\n')

需要在终端运行，notebook会报错
`python3 preprocess.py\
 --corpus_path  corpora/book_review_bert.txt\
 --vocab_path  models/google_zh_vocab.txt\
 --dataset_path  dataset.pt\
 --processes_num 4 --target bert`

In [43]:
!cp -r /wangjin_fix_1/pre_model/pytorch_bert/chinese_roberta_wwm_large_ext_pytorch models/  

`python scripts/convert_bert_from_huggingface_to_uer.py\
 --input_model_path models/chinese_roberta_wwm_large_ext_pytorch/pytorch_model.bin\
 --layers_num 24 --target bert`


In [51]:
! nohup python3 pretrain.py --dataset_path dataset.pt --vocab_path models/google_zh_vocab.txt\
 --pretrained_model_path google_model.bin --config_path models/bert_large_config.json\
 --output_model_path models/output_model.bin --world_size 1 --gpu_ranks 0\
 --embedding word_pos_seg --encoder transformer --mask fully_visible --target bert\
 --total_steps 5000 --save_checkpoint_steps 5000

Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompatible with libgomp.so.1 library.
	Try to import numpy first or set the threading layer accordingly. Set MKL_SERVICE_FORCE_INTEL to force it.


In [52]:
!mkdir bert_large_model

In [2]:
! cp models/chinese_roberta_wwm_large_ext_pytorch/config.json  bert_large_model/config.json
! cp models/chinese_roberta_wwm_large_ext_pytorch/vocab.txt bert_large_model/vocab.txt

In [5]:
! python scripts/convert_bert_from_uer_to_huggingface.py --input_model_path models/output_model.bin-5000\
 --output_model_path bert_large_model/pytorch_model.bin\
 --layers_num 24\
 --target bert

In [6]:
!nvidia-smi

Mon Apr  5 11:58:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   37C    P0    54W / 300W |   2764MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  